In [1]:
from permutationTestC import GrennPdistOpenMP, GrennPdist, coinShiftPdist
from permutationTest import GreenIntCuda, GreenFloatCuda, getDataGreen, PreProcessoinShift, get_p


In [4]:
#from utils2 import *
import time
#from significance_of_mean_cuda import significance_of_mean_cuda

In [5]:
def timePlotSNS(TIMEParallel, TIMEsingleThred, sampleShape,binVar=False, log=False, path=None):
    """Create SNS timeseries-plot"""
    fig, ax = plt.subplots()
    
    sns.set(style="white")
    sns.set_context("talk")
    
    preparePdParallel = preparePandas(TIMEParallel, sampleShape)
    preparePdSingle = preparePandas(TIMEsingleThred, sampleShape, 'Single thread')
    
    data = preparePdParallel + preparePdSingle
    
    pdData = pd.DataFrame(data, columns=['Method', 'time(s)','bins'])
    
    if log:        
        MAX = max(max(TIMEParallel), max(TIMEsingleThred))
        MIN = min(min(TIMEParallel), min(TIMEsingleThred))

        RANGE = np.arange(np.floor(MIN), np.ceil(MAX))
        snsPlot = sns.lineplot(x="bins", y="time(s)",
             hue="Method",
             data=pdData)#.set(yticks = RANGE, yticklabels=10**RANGE)
        plt.yticks(RANGE, 10.0**RANGE)
        
        
    else:
        snsPlot = sns.lineplot(x="bins", y="time(s)",
             hue="Method",
             data=pdData,)
    
    if binVar:
        plt.xlabel(r"$n$", fontsize=20)
        
    else:
        plt.xlabel(r"$n_{w}$", fontsize=20)
    
    plt.ylabel(r"$time(s)$", fontsize=20)
    
    
    
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[1:], labels=labels[1:])
    
    plt.setp(snsPlot.get_legend().get_texts(), fontsize=20)
    plt.xticks(size = 20)
    plt.yticks(size = 20)
    plt.tight_layout()
    
    if path:   
        fig = snsPlot.get_figure()
        fig.savefig(path)

### Get synthetic data

In [ ]:
num_examples = 1
N = [50,100, 150, 200, 250]
s =  [10, 50, 100, 200]
path = "figures/parallelVsSingleThread"

In [ ]:
np.random.seed(3)
n = 10
m = n
n_samples = 5
A = np.asarray([np.random.randint(0,n,m,dtype=np.int32) for _ in range(n_samples)])
B = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)])

In [ ]:
def runExperiment():
    cpu1 = list()
    gpu1 = list()
    n = N[1]
    for bins in s:
        np.random.seed(42)
        A = np.asarray([np.random.randint(0,n,m,dtype=np.int32) for _ in range(n_samples)])
        B = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)])

        start = time.time()
        SGM = significance_of_mean_cuda(bins,dtype_v=np.uint16,dtype_A=np.float64)
        SGM.run(A,B)
        p = SGM.get_p_values()
        end = time.time()
        t_gpu = end - start
        gpu1.append(t_gpu)
        print("GPU: ", t_gpu)
    
    
        start = time.time()
        P = calibration_test(A,B,bins)
        end = time.time()
        t_cpu = end - start
        cpu1.append(t_cpu)
        print("CPU1: ",t_cpu)
    
        print(np.allclose(p,P))
    return cpu1, gpu1
    

In [ ]:
if __name__=="__main__":
    cpu1, gpu1 = runExperiment()
    timePlotSNS(np.array(gpu1), np.array(cpu1), np.array(s), log=False, path=getPATH(path, "nonLog"))
    timePlotSNS(np.log10(np.array(gpu1)), np.log10(np.array(cpu1)), np.array(s), log=True, path=getPATH(path, "Log"))